In [1]:
import os 
import pandas as pd 
import itertools
    

In [86]:
def preprocess(data, info):
    a = data.copy()
    info = re.findall(r".*(?=.csv)",info)[0]
    a = a.rename(columns={"First Tooltip": info})

    """    
    if ("Dim1" in a.columns.values):
        a['Both sexes'] = (a['Dim1'] == "Both sexes").replace({False:0, True:1})
        a['Male'] = (a['Dim1'] == "Male").replace({False:0, True:1})
        a['Female'] = (a['Dim1'] == "Female").replace({False:0, True:1})
    else:
        a['Both sexes'] = (a['Location'] != "Both sexes").replace({False:0, True:1})
        a['Male'] = (a['Location'] != "Male").replace({False:0, True:1})
        a['Female'] = (a['Location'] != "Female").replace({False:0, True:1})
    """

     
    data_2015 = a[a["Period"] == 2015]
    
    data_2016 = a[a["Period"] == 2016]
    
    data_2017 = a[a["Period"] == 2017]
    
    data_2018 = a[a["Period"] == 2018]
    
    data_2019 = a[a["Period"] == 2019]
    
    data_2020 = a[a["Period"] == 2020]
    
    data_2021 = a[a["Period"] == 2021]

    year_dic = {'2015':data_2015,'2016':data_2016,'2017':data_2017,'2018':data_2018,
                '2019':data_2019,'2020':data_2020,'2021':data_2021}
    
    if("Dim1" in a.columns.values):
        for key,df in year_dic.items():
            temp = df.groupby("Location").mean()
            temp.reset_index(inplace=True)
            temp = temp.loc[:, ['Location', info]]
            year_dic[key] = temp
    
    else:
        for key,df in year_dic.items():
            temp = df.loc[:, ['Location', info]]
            year_dic[key] = temp
    return year_dic


In [79]:
def merge_dic(data_lst):

    year_lst = [str(i) for i in range(2015,2022)]
    result = []
    for year in year_lst:
        temp = None
        indicator = False
        for variable_dict in data_lst: 
            if indicator == False:
                temp = variable_dict[year]
                indicator = True
            else:          
                temp = pd.merge(temp, variable_dict[year], on = "Location")
           
        result.append(temp)
    return result

In [80]:
def read_data():
    data_lst = []

    for info in os.listdir("train_data"): 

        if info == ".DS_Store":
            continue
        domain = os.path.abspath(r"train_data")
        dirc = os.path.join(domain,info)
        data = pd.read_csv(dirc)
        
        data_lst.append([data, info])
    return data_lst


In [81]:
def run_proprocess(data_lst):
    result = []
    for [data, info] in data_lst:
        temp = preprocess(data, info)
        result.append(temp)
    return result

In [82]:
def main():
    raw_data = read_data()
    test = []
    comb = []
    result_lst = []
    for L in range(0, len(raw_data)+1):
        for subset in itertools.combinations(raw_data, L):
            if(len(subset) != 0):
                data_lst = subset
                data_lst = run_proprocess(data_lst)
                result = merge_dic(data_lst)
                count = 0
                for i in result:
                    count += i.shape[0]
                test.append([len(subset), count])
                result_lst.append(result)
                comb.append(subset)
                
                
                
    return [test,comb,result_lst]
 

In [72]:
#don't run it if there are more than 15 datasets in train
[test, comb, result] = main()

KeyboardInterrupt: 

In [73]:
test

[[1, 376], [1, 366], [2, 182]]

In [87]:
raw_data = read_data()
data_lst = run_proprocess(raw_data)
data_lst[0]['2015']

,Location,HALElifeExpectancyAtBirth
0,Afghanistan,52.606667
1,Albania,69.046667
2,Algeria,65.996667
3,Angola,53.706667
4,Antigua and Barbuda,66.803333
...,...,...
178,Venezuela (Bolivarian Republic of),65.150000
179,Viet Nam,64.926667
180,Yemen,58.383333
181,Zambia,52.700000


example: output of 2015

In [93]:
temp = None
indicator = False
for variable_dict in data_lst: 
    if indicator == False:
        temp = variable_dict['2015']
        indicator = True
    else:          
        temp = pd.merge(temp, variable_dict['2015'], how ="left",on = "Location")       


In [94]:
temp = temp.rename(columns={"alcoholSubstanceAbuse.csv": "alcoholSubstanceAbuse",
                    "30-70cancerChdEtc.csv": "30-70cancerChdEtc",
                    "birthAttendedBySkilledPersonal.csv": "birthAttendedBySkilledPersonal"})

In [95]:
temp

,Location,HALElifeExpectancyAtBirth,nursingAndMidwife,basicDrinkingWaterServices,alcoholSubstanceAbuse,uhcCoverage,crudeSuicideRates,30-70cancerChdEtc,safelySanitization,medicalDoctors,...,tobaccoAge15,atLeastBasicSanitizationServices,incedenceOfTuberculosis,birthAttendedBySkilledPersonal,mortalityRatePoisoning,maternalMortalityRatio,interventionAgianstNTDs,dentists,pharmacists,dataAvailibilityForUhc
0,Afghanistan,52.606667,1.30,52.39,0.204667,34,4.700000,29.833333,NaN,2.85,...,NaN,44.483333,189 [122-270],50.5,1.230000,701 [501-1020],13765392,0.036,0.5,NaN
1,Albania,69.046667,NaN,89.88,6.730000,58,6.000000,17.333333,39.800000,NaN,...,29.866667,97.623333,17 [14-19],NaN,0.426667,15 [8-25],16,NaN,NaN,NaN
2,Algeria,65.996667,NaN,88.69,0.930000,76,3.233333,14.366667,18.336667,NaN,...,19.266667,86.460000,75 [57-94],NaN,0.800000,114 [67-199],7561,NaN,NaN,NaN
3,Angola,53.706667,NaN,26.71,8.060000,38,5.033333,16.933333,NaN,NaN,...,NaN,43.703333,366 [237-523],NaN,2.960000,251 [178-353],18534219,NaN,NaN,NaN
4,Antigua and Barbuda,66.803333,45.09,96.74,6.016667,73,0.766667,22.600000,NaN,NaN,...,NaN,87.500000,4.3 [3.7-5],100.0,0.376667,43 [25-70],158,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178,Venezuela (Bolivarian Republic of),65.150000,NaN,95.83,6.856667,73,3.800000,18.233333,23.970000,NaN,...,NaN,93.940000,30 [23-38],99.3,0.300000,115 [92-151],280243,NaN,NaN,NaN
179,Viet Nam,64.926667,14.23,90.73,7.950000,73,7.233333,17.500000,NaN,7.96,...,NaN,82.143333,199 [125-290],NaN,0.936667,45 [33-62],4468764,NaN,3.4,NaN
180,Yemen,58.383333,7.24,54.70,0.054333,40,8.566667,30.733333,67.010000,NaN,...,22.166667,63.066667,48 [42-54],NaN,3.733333,169 [105-264],5840695,NaN,NaN,NaN
181,Zambia,52.700000,NaN,40.59,6.946667,54,6.100000,18.066667,NaN,NaN,...,15.433333,26.986667,391 [253-558],NaN,3.066667,232 [178-305],11830416,NaN,NaN,NaN


In [96]:
outputpath= "output.csv"
temp.to_csv(outputpath,sep=',',index=False,header=False)